In [ ]:
from google.auth import default
credentials, project = default()

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()
from tqdm import tqdm

from src.utils import (load_json, get_first_n_items, list_story_folders, save_json, save_text_file, load_text_file)  # noqa: E402
from src.story import upload_story_image, prepare_dialogue_with_wiktionary
from src.gcs_storage import check_blob_exists, read_from_gcs, upload_to_gcs, get_story_translated_dialogue_path, get_story_dialogue_path
from src.config_loader import config
config.TARGET_LANGUAGE_NAME

# Story Data

* based around a story_name which is part of a collection (LM1000 > story_the_birthday_party)
* translated dialogue (text file)
    * comes from google translate
    * added to with Wiktionary links for each utternace ('wiktionary_links')
* audio file for each utterance of Sam and Alex
* 1 x fast audio file for each story part, introduction etc
* an image for each story part

## Storage overview

We want to save the english story dialogue as dialogue.json

Then translations by language_name, then each phrase as an audio clip. We might want higher quality audio for the stories e.g. chirp3 voice

```raw
collections/LM1000/stories/story_murder_mystery/dialogue.json
collections/LM1000/stories/story_murder_mystery/dialogue/french/translated_dialogue.json
collections/LM1000/stories/story_murder_mystery/images/introduction.png etc
collections/LM1000/stories/story_murder_mystery/audio/french/introduction/part_0_sam.mp3
collections/LM1000/stories/story_murder_mystery/audio/french/introduction/part_1_alex.mp3
collections/LM1000/stories/story_murder_mystery/audio/french/introduction/fast/part_1_alex.mp3```


This allows us to quickly retrieve a phrase based on the bucket name and the phrase key, as well as modify individual phrases for later correction

In [ ]:
language_name = config.TARGET_LANGUAGE_NAME.lower()
print(language_name)
config.get_voice_models()

In [ ]:
from src.gcs_storage import get_stories_from_collection

all_stories = get_stories_from_collection(collection="LM1000")

## Loop all stories (text)

Translate and re-upload

In [ ]:
# loop stories and translate
from src.gcs_storage import check_blob_exists, get_story_dialogue_path, get_story_translated_dialogue_path
from src.dialogue_generation import translate_and_upload_dialogue

for story_name in all_stories:
    # get the dialogue
    story_file_path = get_story_dialogue_path(story_name, collection="LM1000")
    translated_file_path = get_story_translated_dialogue_path(story_name, collection="LM1000")
    if check_blob_exists(config.GCS_PRIVATE_BUCKET, translated_file_path):
        print(f"{story_name} already translated")
        continue
    story_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, story_file_path)
    translate_and_upload_dialogue(story_dialogue, story_name, collection="LM1000")

## Add wiktionary links to each story utterance

In [ ]:
for story_name in all_stories[2:]:
    print(f"processing {story_name}")
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection="LM1000")
    if not check_blob_exists(config.GCS_PRIVATE_BUCKET, translated_file_path):
        print(f"{story_name} not yet translated")
        continue
    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    translated_dialogue_with_links = prepare_dialogue_with_wiktionary(translated_dialogue)
    # now re-upload it with embedded witkionary_links
    uploaded = upload_to_gcs(obj=translated_dialogue_with_links, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)
    print(f"uploaded {story_name} : {uploaded}")

Generate audio and upload

In [ ]:
# get the translated dialogue
file_path = f"collections/LM1000/stories/{story_name}/dialogue/{language_name}/translated_dialogue.json"
translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, file_path)

In [ ]:
translated_dialogue

### Generate audio and upload

In [ ]:
config.get_voice_models()

In [ ]:
from src.audio_generation import generate_dialogue_audio_and_upload

generate_dialogue_audio_and_upload(translated_dialogue, story_name)

### Loop through stories to generate audio

In [ ]:
all_stories

In [ ]:
for story_name in all_stories:
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection="LM1000")

    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    generate_dialogue_audio_and_upload(translated_dialogue, story_name)

### Fast Audio 
1 file for each story part

In [ ]:
from src.audio_generation import generate_and_upload_fast_audio

for story_name in tqdm(all_stories):
    # uploads 1 fast mp3 file for each story part. 
    generate_and_upload_fast_audio(story_name, collection="LM1000", overwrite=False)

## Upload images
For when we have them stored locally

In [ ]:
for story_name in all_stories:
    story_dialogue_file = story_dir / story_name / f"{story_name}.json"
    # get the dialogue
    story_file_path = f"collections/LM1000/stories/{story_name}/dialogue.json"
    story_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, story_file_path)
    for story_part in story_dialogue:
        image_file = story_dir / story_name / f"{story_name}_{story_part}.png"
        assert image_file.exists()
        upload_story_image(image_file, story_part, story_name)